In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backpor

## **Deliverable#1**

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-12-26 17:09:41--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2021-12-26 17:09:42 (6.53 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [10]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [35]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/index.txt"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("index.txt"), sep="\t", header=True, inferSchema=True)
df.show(truncate=False)

+---------------------------------------------------------------------------------------------------------+
|SAMPLE CONTENT:                                                                                          |
+---------------------------------------------------------------------------------------------------------+
|https://s3.amazonaws.com/amazon-reviews-pds/tsv/sample_us.tsv                                            |
|https://s3.amazonaws.com/amazon-reviews-pds/tsv/sample_fr.tsv                                            |
|DATA COLUMNS:                                                                                            |
|marketplace       - 2 letter country code of the marketplace where the review was written.               |
|customer_id       - Random identifier that can be used to aggregate reviews written by a single author.  |
|review_id         - The unique ID of the review.                                                         |
|product_id        - The uni

In [36]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/sample_us.tsv"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("sample_us.tsv"), sep="\t", header=True, inferSchema=True)
df.show(truncate=False)

+-----------+-----------+--------------+----------+--------------+----------------------------------------------------------------------------------------+----------------+-----------+-------------+-----------+----+-----------------+-----------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------+
|marketplace|cus

### Create DataFrames to match tables

In [12]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame


In [13]:
# Create the customers_table DataFrame
customers_df = df.groupby("customer_id").agg({"customer_id":"count"}).withColumnRenamed("count(customer_id)", "customer_count")
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   18461411|             1|
|   44331596|             1|
|   20962528|             1|
|   34874898|             1|
|    6762003|             1|
|    1297934|             1|
|   13328687|             1|
|    2749569|             1|
|   12612039|             1|
|   11613707|             1|
|    5543658|             1|
|    7360347|             1|
|   45601416|             1|
|   27225859|             1|
|   32910511|             1|
|   47546726|             1|
|   13545982|             1|
|   12191231|             1|
|     433677|             1|
|   13394189|             1|
+-----------+--------------+
only showing top 20 rows



In [14]:
# Create the products_table DataFrame and drop duplicates. 
products_df = df.select(["product_id","product_title"]).drop_duplicates()
products_df.show(truncate=False)

+----------+---------------------------------------------------------------------------------------+
|product_id|product_title                                                                          |
+----------+---------------------------------------------------------------------------------------+
|B00CAEEDC0|Seat Pets Car Seat Toy                                                                 |
|B00V5DM3RE|Yoga Joes - Green Army Men Toys                                                        |
|B008W1BPWQ|Peppa Pig 7 Wood Puzzles In Wooden Storage Box (styles will vary)                      |
|1591749352|Klutz Sticker Design Studio: Create Your Own Custom Stickers Craft Kit                 |
|B002LHA74O|Super Jumbo Playing Cards by S&S Worldwide                                             |
|B00UZOPOFW|Emazing Lights eLite Flow Glow Sticks - Spinning Light LED Toy                         |
|B004C04I4I|Disney Baby: Eeyore Large Plush                                                

In [15]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
review_id_df = df.select(["review_id","customer_id","product_id","product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.show(truncate=False)

+--------------+-----------+----------+--------------+-----------+
|review_id     |customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|RDIJS7QYB6XNR |18778586   |B00EDBY7X8|122952789     |2015-08-31 |
|R36ED1U38IELG8|24769659   |B00D7JFOPC|952062646     |2015-08-31 |
|R1UE3RPRGCOLD |44331596   |B002LHA74O|818126353     |2015-08-31 |
|R298788GS6I901|23310293   |B00ARPLCGY|261944918     |2015-08-31 |
|RNX4EXOBBPN5  |38745832   |B00UZOPOFW|717410439     |2015-08-31 |
|R3BPETL222LMIM|13394189   |B009B7F6CA|873028700     |2015-08-31 |
|R3SORMPJZO3F2J|2749569    |B0101EHRSM|723424342     |2015-08-31 |
|R2RDOJQ0WBZCF6|41137196   |B00407S11Y|383363775     |2015-08-31 |
|R2B8VBEPB4YEZ7|433677     |B00FGPU7U2|780517568     |2015-08-31 |
|R1CB783I7B0U52|1297934    |B0013OY0S0|269360126     |2015-08-31 |
|R2D90RQQ3V8LH |52006292   |B00519PJTW|493486387     |2015-08-31 |
|R1Y4ZOUGFMJ327|32071052   |B001TCY2DO|459122467     |2015-08-

In [22]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show(truncate=False)

+--------------+-----------+-------------+-----------+----+-----------------+
|review_id     |star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|RDIJS7QYB6XNR |5          |0            |0          |N   |Y                |
|R36ED1U38IELG8|5          |0            |0          |N   |Y                |
|R1UE3RPRGCOLD |2          |1            |1          |N   |Y                |
|R298788GS6I901|5          |0            |0          |N   |Y                |
|RNX4EXOBBPN5  |1          |1            |1          |N   |Y                |
|R3BPETL222LMIM|5          |0            |0          |N   |Y                |
|R3SORMPJZO3F2J|3          |2            |2          |N   |Y                |
|R2RDOJQ0WBZCF6|5          |0            |0          |N   |Y                |
|R2B8VBEPB4YEZ7|5          |0            |0          |N   |Y                |
|R1CB783I7B0U52|1          |0            |1          |N   |Y    

### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [17]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://database-2.c9olawi0alk8.us-east-2.rds.amazonaws.com:5432/postgres"
config = {"user":"postgres", 
          "password": "<hide>", 
          "driver":"org.postgresql.Driver"}

In [18]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [19]:
# Write products_df to table in RDS
# about 3 min
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [20]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [21]:
# Write vine_df to table in RDS
# 11 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)